In [14]:
import pandas as pd
import numpy as np


#reading files
Provider=pd.read_excel("Provider.xlsx", dtype={'Amount': str})
Bill=pd.read_excel("Bill.xlsx", dtype={'amount': str,'billID': str,'billPaymentID': str})
Bill.dtypes
pd.set_option('max_colwidth', None)


In [15]:
#getting info about columns (EDA Phase)

Provider.info()
Provider.drop_duplicates()
Bill.info()
Bill.drop_duplicates()
Bill.providerID.unique()    #'P1', nan, 'P2'
Bill.status.unique()    #'COMPLETED', 'REQUESTED'

Values_COMPLETED=Bill[Bill["status"]=="COMPLETED"]
Values_COMPLETED.groupby("providerID").first()

Values_REQUESTED=Bill[Bill["status"]=="REQUESTED"]
Values_REQUESTED.groupby(["providerID","referenceID"],dropna=False).count()  #for most rows with Requested states>>"providerID","referenceID" are null


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Amount        2919 non-null   object
 1   Ref No        2919 non-null   int64 
 2   Descriptions  2919 non-null   object
dtypes: int64(1), object(2)
memory usage: 68.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45101 entries, 0 to 45100
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   amount         45101 non-null  object
 1   providerID     43557 non-null  object
 2   billID         45101 non-null  object
 3   billPaymentID  45101 non-null  object
 4   referenceID    43557 non-null  object
 5   status         45101 non-null  object
 6   transactionID  45101 non-null  object
dtypes: object(7)
memory usage: 2.4+ MB


amount  billID  billPaymentID  status  transactionID
providerID referenceID                                                      
P2         2721035940        1       1              1       1              1
           2721039609        1       1              1       1              1
           27210410246       1       1              1       1              1
           27210527106       1       1              1       1              1
           27210910237       1       1              1       1              1
           27288023          1       1              1       1              1
           275022692         1       1              1       1              1
           2750371024        1       1              1       1              1
           275040537         1       1              1       1              1
           275043730         1       1              1       1              1
NaN        NaN            1544    1544           1544    1544           1544

In [17]:
#cleaning & integrating data 

#renaming & converting
Provider.rename(columns = {'Ref No':'referenceID'}, inplace = True)
Bill['billPaymentID'] = Bill['billPaymentID'].astype(str)
Bill['billID'] = Bill['billID'].astype(str)
Provider['referenceID'] = Provider['referenceID'].astype(str)

Provider.head(30)
Provider.dtypes

Bill.dtypes
Bill.billID.head(40)

#extracting billID & billpaymentID 
Provider['Numbers'] = Provider['Descriptions'].str.findall('(\d+)').astype(str)
newDesc = Provider["Numbers"].str.split(",", n=1 ,expand = True)
Provider["billID"]=newDesc[0].str.slice(2,-1).astype(str)
Provider["billPaymentID"]=newDesc[1].str.slice(2,-2).astype(str)
Provider=Provider.drop(["Numbers"],axis=1)
Provider



,Amount,referenceID,Descriptions,billID,billPaymentID
0,900,2721063604,‍‍‍پرداخت قبضبا شناسه 4070910357249 و شناسه پرداخت 900733,4070910357249,900733
1,32300,272109585,‍‍‍پرداخت قبضبا شناسه 409580633423 و شناسه پرداخت 3230392,409580633423,3230392
2,75400,27281098610,‍‍‍پرداخت قبضبا شناسه 905102967222510 و شناسه پرداخت 75424950,905102967222510,75424950
3,410800,2721029368,‍‍‍پرداخت قبضبا شناسه 2233358952240 و شناسه پرداخت 41080706,2233358952240,41080706
4,300,2721068678,‍‍‍پرداخت قبضبا شناسه 421049506730 و شناسه پرداخت 325974,421049506730,325974
...,...,...,...,...,...
2914,5710300,2721057653,‍‍‍پرداخت قبضبا شناسه 63039104204355 و شناسه پرداخت 5710305664,63039104204355,5710305664
2915,38600,2721088856,‍‍‍پرداخت قبضبا شناسه 22510809507257 و شناسه پرداخت 3860376,22510809507257,3860376
2916,53900,27210331068,‍‍‍پرداخت قبضبا شناسه 2228361060210 و شناسه پرداخت 53906104,2228361060210,53906104
2917,2900,27210666105,‍‍‍پرداخت قبضبا شناسه 496930442249 و شناسه پرداخت 290742,496930442249,290742


In [18]:
#finding conflicts



#checking replacement values
"""
Provider.rename(columns = {'billPaymentIsD':'billPaymentID'}, inplace = True)
Provider.rename(columns = {'billIsD':'billID'}, inplace = True)
mergedTBL= Provider.merge(Bill, on=["referenceID","billID","billPaymentID"], how='left')
mergedTBL[ (mergedTBL['providerID'].notnull())]
"""
#joining and filtering

Provider.Amount=Provider.Amount.astype(str)
Bill.amount=Bill.amount.astype(str)

mergedTBL= Provider.merge(Bill, on=["referenceID","billID","billPaymentID"], how='left')

x=mergedTBL.groupby("status")
x.count()
mergedTBL[(mergedTBL['status'] == "REQUESTED")]

x=mergedTBL.groupby("providerID")
x.count()
mergedTBL[(mergedTBL['providerID'] == "P1")]
mergedTBL.info()

#filtering

mergedTBL.providerID.unique()
mergedTBL[(mergedTBL['Amount'] != mergedTBL['amount']) & (mergedTBL['providerID'].notnull())]

##there are 2 records that havent same values for amount column >reason> entry error

mergedTBL[(mergedTBL['Amount'] != mergedTBL['amount'])].groupby("status",dropna=False).count()
mergedTBL[(mergedTBL['Amount'] != mergedTBL['amount'])].groupby("providerID",dropna=False).count()
mergedTBL[(mergedTBL['Amount'] != mergedTBL['amount']) & (mergedTBL['status']=="REQUESTED")]

##some records that are not joined

NotJoined=mergedTBL[(mergedTBL['Amount'] != mergedTBL['amount']) & (mergedTBL['providerID'].isnull())].referenceID.unique()
NotJoined = pd.DataFrame(NotJoined, columns=["referenceID"])
NotJoined

NotJoinedMergeTBL= NotJoined.merge(Provider, on="referenceID", how='left')
NotJoinedMergeTBL
NotJoinedMergeTBL= NotJoinedMergeTBL.merge(Bill, on="referenceID", how='left')
#NotJoinedMergeTBL.to_excel("errors.xlsx")

#searching abnormal values
Bill[(Bill['referenceID'] == "076671051010270362")]
          




<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 2918
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Amount         2919 non-null   object
 1   referenceID    2919 non-null   object
 2   Descriptions   2919 non-null   object
 3   billID         2919 non-null   object
 4   billPaymentID  2919 non-null   object
 5   amount         2911 non-null   object
 6   providerID     2911 non-null   object
 7   status         2911 non-null   object
 8   transactionID  2911 non-null   object
dtypes: object(9)
memory usage: 228.0+ KB


,amount,providerID,billID,billPaymentID,referenceID,status,transactionID
1,2900,P1,9501010210430272,2906668,076671051010270362,COMPLETED,fa910101094-c3102-4c68-10647-7d41064e10de88


In [19]:
#finding abnormal amounts
Bill
Bill.amount=pd.to_numeric(Bill['amount'], errors='coerce')
abnormal_amount=Bill[(Bill['amount'] > 1000000000)]
#abnormal_amount.to_excel("abnormal.xlsx")
Bill[(Bill['referenceID'].isnull() & Bill['status']=="COMPLETED")]     #null

#finding duplicated records
newBill=Bill.drop(["transactionID"],axis=1)
d=newBill.groupby(['amount', 'providerID',"billID","billPaymentID","referenceID","status"]).count()
d.duplicated()

Series([], dtype: bool)